In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive/')

# 目的の場所（フォルダ・ディレクトリ）へ移動（各自の環境で適宜修正）
%cd "/content/drive/MyDrive/Colab Notebooks/JKJ1A/"
%ls

---
# 課題



## 課題１

`train_cifar10.py`を作成せよ．`src`に補助用のテンプレートがあるのでそれを完成させよ．

- `lenet.py`にはネットワークの定義を書く
- `cifar10.py`には`load_data()`の定義を書く
- `train_cifar10.py`にはネットワークの訓練を書く
その際，
- 実験時間では学習データ数を制限したが，全てのデータを利用して学習せよ
```
trainloader, testloader, classes = load_data(batch_size, use_all=True)
```
の`use_all=True`のようにすれば良い．

作成した.pyファイルを以下のように実行せよ．

In [ ]:
%%time
!python src/train_cifar10.py --nepochs 2 --batch_size 128 --lr 0.01 --save_model_name 'model/model_cifar10_lenet.pth'

学習したモデルをロードし，訓練データ，テストデータでの精度を計算・表示せよ．

上のコードで学習後に出る精度と一致することを確認せよ．

In [ ]:
# `model_cifar10_day1.pth`をロードしてください．

net = ...


In [ ]:
# test()関数で訓練データ・テストデータでの分類精度を計算・表示してください
import sys
sys.path.append('src')  # srcの中のファイルがimportできるようになる
from train_cifar10 import test
from cifar10 import load_data

...

## 課題２（重要）
**この課題の結果は次回の実験で利用するので必ず行うこと**


VGG11モデルでCIFAR10を学習せよ．課題１のコードを次のように微修正すれば良い．

- `from lenet import Net` -> `from vgg import VGG as Net`
- `net = Net()` -> `net = Net('VGG11')`

学習したモデルの分類精度を課題１の結果と比較せよ．学習したモデル名は`model_cifar10.pth`として`model/`下に保存せよ．テストデータでの分類精度は80%を超えるようにせよ（`nepochs`や`lr`を調整）．友人などと協力して，適当なパラメータを手分けして探すなどすると効率が良い．

## 課題３

CIFAR10ではなくFashionMNISTを学習し，学習のログと分類精度を表示せよ（コードは`train_fmnist.py`とする）．


まず学習の前に，FashionMNISTがどのようなデータを含むか，何枚か画像とラベルを表示させてみよ（`Day1-CIFAR10-classification.ipynb`内のコードを参考にせよ）

In [1]:
# FashionMNISTの画像を何枚か表示



さて，学習のための作業の方針は次のようになる．課題１の`load_data()`を修正し，`fmnist.py`を作る．

- `load_data()`の`transform`を以下のようにする
```
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))]
        )
```

- `torchvision.datasets.CIFAR10`と`classes`も修正する（ほぼ自明な修正．わからなければ少し検索してみよ）．

またモデルは次のモデルを利用せよ（`mlp.py`として保存し，train_fmnist.py内で読み込むこと）．
```
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):   
        super().__init__()
        self.width = 128
        self.fc1 = nn.Linear(28*28, self.width)  # 入力28*28次元, 出力128次元
        self.fc2 = nn.Linear(self.width, self.width)
        self.fc3 = nn.Linear(self.width, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

```

In [ ]:
%%time
#以下を実行できれば良い．
!python src/train_fmnist.py .....

## 課題4 （最終レポート課題）

**この内容は最終レポートに含める課題の一つとする**

モデルの表現力は学習可能なパラメタ数と関係がある．直感的には，学習可能なパラメタ数が多いほど複雑な関数を表現でき，難しい問題が解けると考えられる．次の課題に取り組め．

1. レイヤー数（`nn.Linear`の数）を固定した時，各中間層のユニット数（`self.width`の値）の変化に対し，学習結果はどのようになるだろうか．
2. 逆に，各中間層のユニット数を固定した時，レイヤー数を変化させると学習結果はどのようになるだろうか．
3. 深いネットワーク（レイヤー数が多いネットワーク）と広いネットワーク（各レイヤーのユニット数が多い），どちらが良いだろうか．（ほぼ）同じパラメタ数の場合で比較せよ．

文献調査を行い，どのような傾向があるか知った上でそれを再現しても良い．その場合は参考文献を示すこと．

---

*補足（1に関して）：課題3の`mlp.py`に定義したモデルに関して，ネットワークの幅`self.width`を何パターンか変化させ，FashionMNIST学習後の分類精度をプロットする（横軸`width`, 縦軸は分類精度）．*

*補足（2に関して）：課題3の`mlp.py`を参考に異なるレイヤー数のモデルを定義することになる．*

*補足（3に関して）：
`nn.Linear`は全結合層と呼ばれ，入力$x$に対し変換$Wx+b$を行い$z$を出力する（$W,b$は学習されるパラメタ）．$x,z$がそれぞれ$n,m$次元ベクトルの場合，全結合層一つ当たりのパラメタ数が$n,m$で表現できると思う．「同じパラメタ数」という条件が何を意味するのかよく考えること．*

*補足：
`plt.show()`の一つ前で`plt.savefig('画像名.pdf')`とすると，画像を保存できる．*

```
import matplotlib.pyplot as plt 

parameter = ...
accs = ...

plt.plot(parameter, accs)
plt.savefig('result.pdf')
plt.show()
```